<a href="https://colab.research.google.com/github/tomonari-masada/course2024-stats1/blob/main/03_text_retrieval_with_multinomial_distributions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 多項分布を使った文書検索

## 説明
* 検索対象の各文書について、最尤推定で単語確率を求める。
  * $\phi_{d,w}$: 文書$d$における単語$w$の出現確率
* クエリの尤度を、各文書について求めた単語確率を使って計算する。
  * $n_{q,w}$: クエリ$q$における単語$w$の出現頻度
  * このとき、文書$d$の単語確率を使ったクエリ$q$の対数尤度は、以下の通り。
$$\begin{align}
L_q(d) = \sum_w n_{q,w} \log \phi_{d,w}
\end{align}$$
  * 上の式で、規格化定数の部分は省略している。（ランキングに関係しないため。）
* このように計算されたクエリの尤度によって、検索対象の文書をソートする。
  * $L_q(d)$が大きい順に、文書を検索結果として表示する。
* 上記の方法では検索があまりうまくいかないことを確認する。
  * 上の式を使うと、検索対象の文書に出現しない単語を含むクエリの尤度はゼロ（対数尤度はマイナス無限大）になる。

## 準備

In [ ]:
import numpy as np
from scipy.stats import multinomial
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

## データセット

* 20 newsgroupsコーパスを使う。
* テストセットの文書一件一件を、クエリだと思う。
* そして、訓練セットの文書の中から、類似文書を検索する。

### データセットのダウンロード

In [ ]:
train_corpus, train_labels = fetch_20newsgroups(subset="train", return_X_y=True)
test_corpus, test_labels = fetch_20newsgroups(subset="test", return_X_y=True)

In [ ]:
print(train_corpus[0])

In [ ]:
len(train_corpus), len(test_corpus)

### 各文書での単語の出現回数を数える

* ここでは、訓練データで20文書未満にしか出現しない単語と、英語のストップワードとを、無視する。

In [ ]:
vectorizer = CountVectorizer(min_df=20, stop_words="english")
X_train = vectorizer.fit_transform(train_corpus).toarray()
X_test = vectorizer.transform(test_corpus).toarray()

In [ ]:
X_train.shape, X_test.shape

### 語彙集合を取得する

In [ ]:
vocabulary = vectorizer.get_feature_names_out()
print(vocabulary)

## 最尤推定

* 各文書を多項分布でモデリングする。
* そして、最尤推定により、単語確率パラメータの値を推定する。

In [ ]:
X_train_probs = X_train / X_train.sum(axis=1).reshape(-1, 1)

In [ ]:
X_train_probs.sum(axis=1)

## 対数尤度を求めるヘルパ関数

* テスト文書の対数尤度を、特定の訓練文書の単語確率を使って求めるヘルパ関数
  * 訓練文書にない単語を含むテスト文書は、対数尤度がマイナス無限大になる
  * クエリと共通する単語がない文書は、検索の対象にならなくなってしまう。

In [ ]:
def log_likelihood(x_test, x_train_prob):
  # 多項分布の作成
  rv = multinomial(x_test.sum(), x_train_prob)
  # 対数尤度を確率質量関数を使って計算
  return rv.logpmf(x_test)

## 検索の実行

* クエリとして使うテスト文書の設定

In [ ]:
query_idx = 100

* クエリ文書の内容を確認

In [ ]:
print(test_corpus[query_idx])

 * 個々の訓練文書ごとに、クエリ文書の対数尤度を計算

In [ ]:
scores = list()
for i in range(len(X_train)):
  scores.append(log_likelihood(X_test[query_idx], X_train_probs[i]))
scores = np.array(scores)

# 降順にソート
sorted_train_indices = (- scores).argsort()

* 実は、このままだと、すべての類似度が`-inf`になる。

In [ ]:
(scores[sorted_train_indices] != - np.inf).sum()

* -infということは、確率ゼロということ。
* つまり、いずれの検索対象の文書も、クエリの確率をゼロにしているということ。
  * ちゃんと検索できていない、ということ。
  * クエリに含まれる単語を全て含む訓練文書が、一つもない、ということ。

* 出現しないアイテムの確率をゼロにする最尤推定は、問題がありそう・・・。
* どう対処すればいいのか？

## スムージング

* 検索性能を上げる一つのテクニック。
* 検索対象の各文書について、単語確率を推定する前に、一律、小さな値を出現回数に加算する。

In [ ]:
# 0.01 という値の部分は、実際には、要チューニング。
X_train = X_train + 0.01
X_train_probs = X_train / X_train.sum(axis=1).reshape(-1, 1)

## 検索の実行

In [ ]:
# 個々の訓練文書ごとに、クエリ文書のスコアを計算
scores = list()
for i in range(X_train.shape[0]):
  scores.append(log_likelihood(X_test[query_idx], X_train_probs[i]))
scores = np.array(scores)

# 降順にソート
sorted_train_indices = (- scores).argsort()

In [ ]:
print(test_corpus[query_idx])

In [ ]:
print(train_corpus[sorted_train_indices[0]])

In [ ]:
test_labels[query_idx], train_labels[sorted_train_indices[0]]

In [ ]:
print(scores[sorted_train_indices[0]])

* -infにはなっていないようだ。

# 課題
スムージングを使った場合、テストセットの文書10件ぐらいについて、検索結果1位の訓練文書が、同じカテゴリに属しているかどうか、チェックする。